In [23]:
import pandas as pd

**Import and prepare the data**

In [24]:
def prepare_data(coin):
    df = pd.read_csv(f'../data/coin_{coin}.csv', index_col=0).reset_index()
    df.Date = pd.to_datetime(df.Date)
    df['dayofweek'] = df.Date.dt.dayofweek
    df['dayofmonth'] = df.Date.dt.day
    df['year_month'] = df.Date.dt.strftime("%Y-%m")
    df['avg_daily_price']= (df.High + df.Low) / 2
    return df

In [25]:
def calculate_profit_dca(df, frequency, purchase_year_month = '2018-01',\
                        purchase_day_of_week = 0, purchase_day_of_month = [0]):
    purchase_usd = 10
    if frequency == 'monthly':
        freq_cond = (df.dayofmonth.isin(purchase_day_of_month))
    if frequency == 'weekly':
        freq_cond = (df.dayofweek == purchase_day_of_week)

    purchase = df[(df.year_month >= purchase_year_month) & freq_cond]
    purchased_amount = (purchase_usd/purchase.High).sum()
    investment_amount = purchase_usd * len(purchase)
    investment_value = df[-1:].High * purchased_amount
    return (investment_value / investment_amount)

In [26]:
dfs = {}
coins = ['Bitcoin', 'Ethereum']

**Monthly**

In [27]:
days= []
monthly_profit = {}
for i in range(1,32):
    days.append(i)

for coin in coins:
    df = prepare_data(coin)
    monthly_profit[coin] = []
    print(f"Calculating profit for coin {coin}...")
    for i in range(1,32):    
        monthly_profit[coin].append(calculate_profit_dca(df, 'monthly', purchase_day_of_month=[i]).iloc[0])

Calculating profit for coin Bitcoin...
Calculating profit for coin Ethereum...


In [28]:
monthly_profit['days']=days
monthly_df = pd.DataFrame.from_dict(monthly_profit).set_index('days')
dfs['monthly'] = monthly_df
monthly_df.head()

,Bitcoin,Ethereum
days,,
1,4.043401,8.670210
2,3.992209,8.522013
3,3.956897,8.473842
4,3.987841,8.543198
5,3.992413,8.512712


**Weekly**

In [29]:
days= []
weekly_profit = {}
for i in range(0,7):
    days.append(i)

for coin in coins:
    df = prepare_data(coin)
    weekly_profit[coin] = []
    print(f"Calculating profit for coin {coin}...")
    for i in range(0,7):    
        weekly_profit[coin].append(calculate_profit_dca(df, 'weekly', purchase_day_of_week=i).iloc[0])

Calculating profit for coin Bitcoin...
Calculating profit for coin Ethereum...


In [30]:
weekly_profit['days']=days
weekly_df = pd.DataFrame.from_dict(weekly_profit).set_index('days')
dfs['weekly'] = weekly_df
weekly_df.head()

,Bitcoin,Ethereum
days,,
0,4.049776,8.685145
1,4.055809,8.732725
2,4.061459,8.746297
3,4.059354,8.754766
4,4.087846,8.822309


**Twice a month**

In [31]:
days= []
twice_a_month_profit = {}
for i in range(1,32):
    for j in range(i+1,32):
        days.append(f"{i}-{j}")

for coin in coins:
    df = prepare_data(coin)
    twice_a_month_profit[coin] = []
    print(f"Calculating profit for coin {coin}...")
    for i in range(1,32):
        for j in range(i+1,32):    
            twice_a_month_profit[coin].append(calculate_profit_dca(df, 'monthly', purchase_day_of_month=[i,j]).iloc[0])

Calculating profit for coin Bitcoin...
Calculating profit for coin Ethereum...


In [32]:
twice_a_month_profit['days']=days
twice_a_month_df = pd.DataFrame.from_dict(twice_a_month_profit).set_index('days')
dfs['twice_a_month'] = twice_a_month_df
twice_a_month_df.head()

,Bitcoin,Ethereum
days,,
1-2,4.017805,8.596112
1-3,4.000149,8.572026
1-4,4.015621,8.606704
1-5,4.017907,8.591461
1-6,4.013217,8.564750


In [33]:
coin_describe_values = {}
for coin in coins:
    describe_values = {}
    for index, df in dfs.items():
        describe_values[index] = df.describe()[coin].values
    coin_describe_values[coin] = describe_values

In [34]:
coin_describe_dfs = {}
for coin in coins:
    coin_describe_dfs[coin] = pd.DataFrame\
                              .from_dict(coin_describe_values[coin], orient='index', columns=df.describe().index)

In [35]:
coin_describe_dfs['Bitcoin'].transpose()

,monthly,weekly,twice_a_month
count,31.000000,7.000000,465.000000
mean,4.068759,4.067256,4.067810
std,0.054852,0.014194,0.037295
min,3.956897,4.049776,3.969964
25%,4.041394,4.057582,4.044567
50%,4.060084,4.061459,4.068969
75%,4.108346,4.078274,4.093880
max,4.159624,4.087846,4.153271


In [36]:
coin_describe_dfs['Ethereum'].transpose()

,monthly,weekly,twice_a_month
count,31.000000,7.000000,465.000000
mean,8.759797,8.752661,8.755408
std,0.172773,0.042145,0.115264
min,8.459290,8.685145,8.466566
25%,8.634049,8.739511,8.675913
50%,8.765556,8.746926,8.759827
75%,8.896741,8.767613,8.835661
max,9.123020,8.822309,9.052571


* The montly DCA has provided the highest profits, but presents also the lowest profits. Indeed its standard deviation is the highest
* The weekly approach has provided more stable profits, regardless of the day picked for investing, but presents a lower maximum profits compared to the other approaches
* In general it seems that the higher the frequency of the DCA, the less influent the day on which the investment is made and the more stable profits. The lower the frequency, the higher the maximum possible profits, but also higher standard deviation and the risk of obtaining lower profits if not picking the right day to invest

We conclude that the best approach for those aiming at maximising the profit should use a monthly approach. As long as the right day in the month is picked to invest, since the risk is to lower the profit if the wrong day is picked. A following analysis will try to identify the best day of month to invest for a specific coin.
For a more stable profit, weekly seems to provide more constant profits, regardless of the days on which the investment is made.